[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/axr/blob/master/axr/00_intro.ipynb)

## Aprendizaje por refuerzo


## Elementos del axr


Además del agente y del entorno, existen cuatro subelementos esenciales en un sistema de axr:

- **Política**: define el comportamiento de el agente en cada momento. Relaciona el estado que el agente percibe de su entorno con todas las posibles acciones que puede tomar. Puede ser tan simple como una función o una tabla, o tan complicada como un proceso de búsqueda. La política define completamente el comportamiento de un agente.
- **Recompensa**: define el objetivo del problema, y es un un valor numérico que en cada momento el entorno envía al agente, el cual tiene el único objetivo de maximizarlo. Es el valor principal a tener en cuenta a la hora de alterar la política de un agente.
- **Función de valor**: mientras que la recompensa indica la calidad de un estado de manera inmediata, la función de valor indica la calidad a largo plazo. El valor de un estado es la cantidad total de recompensa que un agente espera acumular en el futuro empezando en ese mismo estado. De esta manera, estados con una baja recompensa en relación a otros puede ser preferible si su valor es mayor (los estados futuros a los que nos permite llegar proporcionarán mayor recompensa). Así pues, favoreceremos acciones que impliquen mayor valor sobre recompensas inmediatas. El pricipal problema será estimar estos valores, ya que para ellos nuestro agente deberá explorar de manera repetida el entorno actualizando su información para poder llegar a ser capaz de tomar decisiones con fundamento.
- **Modelo** del entorno: imita el comportamiento del entorno y sirve para planificar acciones considerando estados futuros que todavía no se han experimentado. El uso de un modelo del entorno nos permite dividir los métodos de axr en métodos sin modelos, en el que un agente aprende por prueba y error (lo opuesto a la planificación) y métodos con modelo, aunque en varias aplicaciones esta línea es un poco difusa ya que se combinan ambas opciones.

In [ ]:
import numpy as np

# Inicializa una matriz de 6x7
class Board():
    def __init__(self):
        self.state = np.zeros((6, 7))

    # Función que devuelve todos los posibles movimientos válidos (casillas libres)
    def valid_moves(self):
        return [(i, j) for j in range(7) for i in range(6) if self.state[i, j] == 0]

    # Asignar x u o (1 y -1), actualizar el tablero
    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("Movimiento ilegal!")

    # Función para comprobar si ya se ha terminado el juego, comprueba si hay 4 unos en vertical, horizontal o diagonal
    def is_game_over(self):
        # Comprobar filas y columnas
        for r in range(6):
            for c in range(4):
                if abs(sum(self.state[r, c:c+4])) == 4:
                    return 1 if self.state[r, c] == 1 else -1
        for c in range(7):
            for r in range(3):
                if abs(sum(self.state[r:r+4, c])) == 4:
                    return 1 if self.state[r, c] == 1 else -1
        # Comprobar diagonales
        for r in range(3):
            for c in range(4):
                if abs(sum(self.state[r+i, c+i] for i in range(4))) == 4:
                    return 1 if self.state[r, c] == 1 else -1
                if abs(sum(self.state[r+3-i, c+i] for i in range(4))) == 4:
                    return 1 if self.state[r+3, c] == 1 else -1
        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    # Vuelve a poner a 0 el estado para la siguiente iteración, resetea el tablero
    def reset(self):
        self.state = np.zeros((6, 7))


In [ ]:
from tqdm import tqdm

# Clase a la que le pasamos los dos jugadores y asignamos 1 y -1
class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    # Le pasamos número de rondas y agarra a los dos agentes
    def selfplay(self, rounds=100):
        wins = [0, 0] #contar victorias de cada jugador
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over: # Bucle, cada player llama a su función move, los agentes se guardan los estados
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward() #asigna recompensar al final de cada juego
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    #función Asigna la recompensa
    def reward(self):
        winner = self.board.is_game_over() #comrpibar resultado del juego
        if winner == 0: # Empate
            for player in self.players:
                player.reward(0.5)
        else: # Le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [ ]:
# Implementación de la clase agente, alpha es el learning rate, hiperparámetro
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # Tabla con pares estado -> valor, le asigna valor
        self.alpha = alpha         # Learning rate
        self.positions = []       # Guardamos todas las posiciones de la partida, lista
        self.prob_exp = prob_exp   # Probabilidad de explorar, probar acciones aleatorias con una prob de 0.5 (50%) agarrar la mejor acción hasta el momento

    def reset(self):
        self.positions = []
####################################################### 1. Politica, define el comportamiento del agente
    # Función para elegir el proximo moviento #implemntado aqui
    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # Exploración: elige movimiento aleatorio
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # Vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # Explotación, acciones buenas y malas, elijo la mejor
        # Vamos a la posición con más valor
        max_value = -1000
        best_move = None
        for row, col in valid_moves: # Hacemos todos los movimientos válidos
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(6*7))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state) # No ha llegado a esa acción, no la tiene todavía
            if value >= max_value:
                max_value = value
                best_move = (row, col)
        return best_move

    def update(self, board):
        self.positions.append(str(board.state.reshape(6*7)))
############################################### 2. Recompensa, feedback inmediato
############################################3#  3. Función de valor, value_function, diccionario que mapea cada estado del tablero, valor numerico
    # Cada agente llama a esta función, iteramos por todas las posiciones
    def reward(self, reward):
        # Al final de la partida (cuando recibimos la recompensa)
        # Iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]


In [ ]:
# Entrenamiento
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(3000)

100%|██████████| 3000/3000 [07:32<00:00,  6.63it/s]


[1596, 1397]

In [ ]:
import pandas as pd
# Saca la función de valor del agente 1 y lo convierte en tabla
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})
tabla

,estado,valor
0,[ 0. 0. 1. 0. 0. -1. -1. -1. 0. 0. 0. ...,0.5
1,[-1. 0. 0. 0. -1. 0. 0. 0. 0. 0. 0. ...,0.5
2,[ 0. 1. 0. 1. 1. -1. -1. 0. 0. 0. 1. ...,0.5
3,[ 0. 0. 0. 0. 0. 1. -1. 0. -1. 0. -1. ...,0.5
4,[ 1. 0. 0. 1. 1. -1. 1. 0. -1. 1. 1. -...,0.5
...,...,...
66460,[ 0. 0. 0. 0. 0. 0. -1. 0. 0. 0. 0. ...,0.0
66461,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.0
66462,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.0
66463,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,0.0


In [ ]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)